In [83]:
import numpy as np
from model.layers import Embedding, Sigmoid, Softmax, Linear
from utils import _Huffman_Tree, corpus_making, delete_low_freq, train_data_gen

In [75]:
import numpy as np
from model.layers import Embedding, Sigmoid, Softmax, Linear
from utils import _Huffman_Tree

class BCELoss:
    def __init__(self):
        self.params = None
        self.grads = None
        self.eps = 1e-8

        self.y_pred , self.target = None, None
        self.loss = None

    def forward(self, y_pred, target, dim = 1):

        self.y_pred = y_pred
        self.target = target
        self.dim = dim
        number = target.shape[0]
        
        self.loss = -self.target * np.log(self.y_pred + self.eps) - (1 - self.target) * np.log(1 - self.y_pred + self.eps)
        self.loss = np.sum(self.loss, axis = dim)/number

        return self.loss

    def backward(self, dout = 1):
        dx = (self.y_pred - self.target) / (self.y_pred * (1 - self.y_pred) + + self.eps) 
        return dx * dout

class Hsoftmax:
    def __init__(self, vocab_size, projection, sample_size):
        self.Embedding = Embedding(vocab_size, projection)
        self.HSvector = Embedding(vocab_size - 1 , projection)
        self.sigmoid = Sigmoid()
        self.sample_size = sample_size

        self.layers = [self.Embedding, self.HSvector]

        self.params = []
        self.grads = []

        for layer in self.layers:
            self.params.append(layer.params)
            self.grads.append(layer.grads)

    def forward(self, x, label):
        '''
        inputs : 1 x D(projection)
        label : 1 x [direction_path(1, depth), idx_path(1, depth)]
        label 과 output 의 argmax를 비교해서 같으면 1 틀리면 0 을 부여한 후 이를 target vector로 설정해야됨
        '''

        dir_path = np.array(label[0])
        idx_path = np.expand_dims(label[1], 1)
        self.x = x
        
        self.hidden = self.Embedding.forward(x)

        self.hirearchy_vectors = self.HSvector.forward(dir_path)

        out = np.matmul(self.hirearchy_vectors , self.hidden.T )

        out = self.sigmoid.forward(out)

        mask = np.zeros_like(out)
        mask[mask >= 0.5] = 1

        target = np.zeros_like(out)
        target[mask == idx_path] = 1

        return out , target

    def backward(self, dout):

        W_in, W_out = self.params
        #length x 1
        d_sig = self.sigmoid.backward(dout)

        #vocab -1 x hidden
        d_lin = np.matmul(d_sig , self.hidden)
        d_h = np.matmul(dout.T, self.hirearchy_vectors)

        self.HSvector.backward(d_lin)
        self.Embedding.backward(d_h)

In [84]:
path = "./data/text8.txt"

In [112]:
word2idx , idx2word = corpus_making(path, batch= 50000)
word2idx, idx2word = delete_low_freq(word2idx, idx2word, 8)

MAKING CORPUS: 100%|██████████████████████████████████████████████████████████████| 2000/2000 [00:13<00:00, 148.82it/s]


In [87]:
train_set_idx, total_len = train_data_gen(path = path, word2idx = word2idx, max_distance= 3, batch = 50000)

Making Train_set: 100%|████████████████████████████████████████████████████████████| 2000/2000 [00:41<00:00, 47.77it/s]


In [6]:
b_tree, max_ = _Huffman_Tree(word2idx)

Huffman_Tree: 100%|██████████████████████████████████████████████████████████| 47158/47158 [00:00<00:00, 194584.73it/s]


In [76]:
total_num = len(word2idx)
model = Hsoftmax(total_num, 200, 3)
criterion = BCELoss()

In [8]:
label = np.array([[path, idx_path] for _, _, idx_path ,path in b_tree])

In [53]:
batch_train = train_set_idx[np.random.choice(100, 1)]

x_train = batch_train[:,3]
label_train_idx = np.delete(batch_train , 3)

#N x [path(2C), idx_path(2C)]
label_train = label[np.random.choice(label_train_idx, 3)]

In [77]:
for x, lab in zip(repeat(x_train), label_train):
    print(x)
    x = [x]
    y, t = model.forward(x, lab)
    print(t.shape)
    loss = criterion.forward(y, t, dim = 0)
    print(loss)
    dloss = criterion.backward()
    model.backward(dloss)

[16]
(12, 1)
[9.21034037]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
[16]
(17, 1)
[8.66855564]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0.

In [55]:
label_train

array([[array([47157, 47155, 47152, 47145, 47132, 47107, 47065, 46991, 46859,
       46609, 46149, 45345]),
        array([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1])],
       [array([47157, 47155, 47152, 47146, 47134, 47112, 47075, 47004, 46881,
       46647, 46218, 45467, 44196, 42198, 39199, 34931, 29140]),
        array([0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1])],
       [array([47157, 47156, 47154, 47149, 47139, 47121, 47091, 47035, 46936,
       46754, 46417]),
        array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1])]], dtype=object)

In [58]:
from itertools import repeat

In [88]:
len(train_set_idx)

16995207

In [80]:
16.5 / 2

8.25

In [94]:
import os
import zipfile
import collections
import math
import os
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf


url = 'http://mattmahoney.net/dc/'


In [95]:
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [104]:
def read_data(filename):
    ''' zip 파일에 포함된 텍스트 파일을 읽어서 단어 리스트 생성. 포함된 파일은 1개. 
    zip 파일은 30mb, txt 파일은 100mb. '''
    with zipfile.ZipFile(filename) as f:
        names = f.namelist()                # ['text8']
        contents = f.read(names[0])         # 크기 : 100,000,000바이트
        text = tf.compat.as_str(contents)   # 크기 : 100,000,000
        return text.split()                 # 갯수 : 17005207


vocabulary = read_data(filename)
print('Data size', len(vocabulary))         # 17005207

Data size 17005207


In [105]:
# Step 2: 사전을 구축하고 거의 등장하지 않는 단어를 UNK 토큰으로 대체.
# UNK는 unknown 약자로 출현 빈도가 낮은 단어들을 모두 대체한다. UNK 갯수는 418391.
vocabulary_size = 50000


def build_dataset(words, n_words):
    # count : [['UNK', -1], ('the', 1061396), ('of', 593677), ('and', 416629), ...]
    # 크기는 50,000개. UNK가 들어 있고, -1을 뺐으니까 처음에 전달된 크기 사용.
    # 빈도가 높은 5만개 추출.
    # count에 포함된 마지막 데이터는 ('hif', 9). 9번 나왔는데 드물다고 얘기할 수 있는지는 의문.
    unique = collections.Counter(words) # 중복 단어 제거
    print(unique)
    orders = unique.most_common(n_words - 1)        # 단어에 대한 빈도 계산. 갯수를 지정하지 않으면 전체 계산.
    count = [['UNK', -1]]
    count.extend(orders)

    dictionary = {}
    for word, _ in count:
        dictionary[word] = len(dictionary)


    data = []
    for word in words:
        if word in dictionary:          # word가 dictionary에 존재한다면
            index = dictionary[word]
        else:
            index = 0                   # UNK는 0번째에 위치
            count[0][1] += 1            # 갯수 : 418391
        data.append(index)


    return data, count, list(dictionary.keys())

data, count, ordered_words = build_dataset(vocabulary, vocabulary_size)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [165]:
len(data)

17005207

In [164]:
data

[5234,
 3081,
 12,
 6,
 195,
 2,
 3134,
 46,
 59,
 156,
 128,
 742,
 477,
 10572,
 134,
 1,
 27350,
 2,
 1,
 103,
 855,
 3,
 1,
 15068,
 0,
 2,
 1,
 151,
 855,
 3581,
 1,
 195,
 11,
 191,
 59,
 5,
 6,
 10713,
 215,
 7,
 1325,
 105,
 455,
 20,
 59,
 2732,
 363,
 7,
 3673,
 1,
 709,
 2,
 372,
 27,
 41,
 37,
 54,
 540,
 98,
 12,
 6,
 1424,
 2758,
 19,
 568,
 687,
 7089,
 1,
 248,
 5234,
 11,
 1053,
 28,
 1,
 321,
 249,
 44612,
 2878,
 793,
 187,
 5234,
 12,
 6,
 201,
 603,
 11,
 1,
 1135,
 20,
 2622,
 26,
 8984,
 3,
 280,
 32,
 4148,
 142,
 60,
 26,
 6438,
 4187,
 2,
 154,
 33,
 363,
 5234,
 37,
 1138,
 7,
 448,
 345,
 1819,
 20,
 4861,
 1,
 6754,
 2,
 7574,
 1775,
 567,
 1,
 94,
 1,
 248,
 11065,
 12,
 52,
 7089,
 90,
 27,
 271,
 38,
 5949,
 4862,
 20300,
 29,
 0,
 42,
 318,
 6,
 25637,
 528,
 7574,
 372,
 5,
 259,
 2,
 154,
 26,
 1207,
 12,
 7574,
 201,
 1577,
 3,
 15201,
 333,
 1775,
 7089,
 4861,
 345,
 765,
 161,
 407,
 5691,
 756,
 2,
 4106,
 1132,
 4332,
 1537,
 3,
 568,
 8118,
 99

In [172]:
# Step 3: skip-gram 모델에 사용할 학습 데이터를 생성할 함수 작성

def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    ''' Stochastic Gradient Descent 알고리즘에 사용할 minibatch 생성.
    :param data : 단어 인덱스 리스트
    :param batch_size : SGD 알고리즘에 적용할 데이터 갯수. 한 번에 처리할 크기.
    :param num_skips : context window에서 구축할 (target, context) 쌍의 갯수.
    :param skip_window : skip-gram 모델에 사용할 윈도우 크기.
         1이라면 목표 단어(target) 양쪽에 1개 단어이므로 context window 크기는 3이 된다. (단어, target, 단어)
         2라면 5가 된다. (단어 단어 target 단어 단어)
    :param data_index : 첫 번째 context window에 들어갈 data에서의 시작 위치.
    '''

    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    temp = 'batch_size {}, num_skips {}, skip_window {}, data_index {}'

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    # span은 assert에 기술한 코드 때문에 항상 num_skips보다 크다.
    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    # deque
    # 처음과 마지막의 양쪽 끝에서 일어나는 입출력에 대해 가장 좋은 성능을 내는 자료구조
    # maxlen 옵션이 없으면 크기 제한도 없고, 있다면 지정한 크기만큼만 사용 가능.
    # maxlen을 3으로 전달하면 3개만 저장할 수 있고, 새로운 요소를 추가하면 반대쪽 요소가 자동으로 삭제됨.
    # 여기서는 자동 삭제 기능 때문에 사용.
    # data_index 번째부터 span 크기만큼 단어 인덱스 저장
    # 첫 번째 context 윈도우 구성
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    for i in range(batch_size // num_skips):

        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]
            print(targets[j], '**')     # (2, 0), (0, 2), (0, 2), (0, 2)

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index


In [173]:
batch, labels, data_index = generate_batch(data, batch_size=15, num_skips=5, skip_window=5, data_index=0)

7 **
4 **
8 **
9 **
6 **
2 **
6 **
1 **
7 **
0 **
6 **
9 **
7 **
1 **
8 **


In [175]:
labels

array([[  46],
       [ 195],
       [  59],
       [ 156],
       [3134],
       [   6],
       [  46],
       [  12],
       [  59],
       [3081],
       [  59],
       [ 742],
       [ 156],
       [   6],
       [ 128]])

In [170]:
data_index

3

In [171]:
batch

array([   2,    2,    2,    2,    2, 3134, 3134, 3134, 3134, 3134,   46,
         46,   46,   46,   46])

In [160]:
filename = "./text8.zip"
with zipfile.ZipFile(filename) as f:
    names = f.namelist()                # ['text8']
    contents = f.read(names[0])         # 크기 : 100,000,000바이트
    #text = tf.compat.as_str(contents)   # 크기 : 100,000,000
    a = contents.split()
    

In [161]:
with open("./data/")

[b'anarchism',
 b'originated',
 b'as',
 b'a',
 b'term',
 b'of',
 b'abuse',
 b'first',
 b'used',
 b'against',
 b'early',
 b'working',
 b'class',
 b'radicals',
 b'including',
 b'the',
 b'diggers',
 b'of',
 b'the',
 b'english',
 b'revolution',
 b'and',
 b'the',
 b'sans',
 b'culottes',
 b'of',
 b'the',
 b'french',
 b'revolution',
 b'whilst',
 b'the',
 b'term',
 b'is',
 b'still',
 b'used',
 b'in',
 b'a',
 b'pejorative',
 b'way',
 b'to',
 b'describe',
 b'any',
 b'act',
 b'that',
 b'used',
 b'violent',
 b'means',
 b'to',
 b'destroy',
 b'the',
 b'organization',
 b'of',
 b'society',
 b'it',
 b'has',
 b'also',
 b'been',
 b'taken',
 b'up',
 b'as',
 b'a',
 b'positive',
 b'label',
 b'by',
 b'self',
 b'defined',
 b'anarchists',
 b'the',
 b'word',
 b'anarchism',
 b'is',
 b'derived',
 b'from',
 b'the',
 b'greek',
 b'without',
 b'archons',
 b'ruler',
 b'chief',
 b'king',
 b'anarchism',
 b'as',
 b'a',
 b'political',
 b'philosophy',
 b'is',
 b'the',
 b'belief',
 b'that',
 b'rulers',
 b'are',
 b'unnecessa